# Final Project - Part 2

## 1. Title: “CareerChat: Your AI Job Outreach Assistant”

Team:
1.   Anshika Bajpai
2.   Brendan Kelly
3.   Cassie Cagwin

11/9/2025

## 2. Preprocessing 30pts

---

Provide all essential steps that you deem necessary for your application

In [ ]:
##Ask if they'd like to upload their resume or fill out a form with their profile
profile_creation_method = 'Yes, I would like to upload my resume.' #@param ['Yes, I would like to upload my resume.', 'No, I would like to fill out the profile form.']
print(f"Selection: {profile_creation_method}")

In [ ]:
##Profile info imports
##Name, work experience, education, skills, career interest
import ipywidgets as widgets
from IPython.display import display, clear_output

if profile_creation_method == 'No, I would like to fill out the profile form.':

  name_input = widgets.Text(
      description='Name:',
      placeholder='Enter your full name',
      layout=widgets.Layout(width='auto')
  )


  email_input = widgets.Text(
      description='Email:',
      placeholder='user@example.com',
      layout=widgets.Layout(width='auto')
  )

  experience_input = widgets.Textarea(
      description='Work Experience:',
      placeholder='List companies, roles, and achievements (one per line)',
      rows=5, # Number of visible rows
      layout=widgets.Layout(width='auto')
  )

  education_input = widgets.Textarea(
      description='Education:',
      placeholder='List degrees, institutions, and dates',
      rows=3,
      layout=widgets.Layout(width='auto')
  )

  skills_input = widgets.Textarea(
      description='Skills:',
      placeholder='List your key skills (e.g., Python, SQL, NLP)',
      rows=3,
      layout=widgets.Layout(width='auto')
  )

  interest_input = widgets.Textarea(
      description='Career Interests:',
      placeholder='Please describe your career interests',
      rows=3,
      layout=widgets.Layout(width='auto')
  )

  submit_button = widgets.Button(description="Submit Profile", button_style='success')
  output_area = widgets.Output()

  def on_submit_click(b):
      with output_area:
          clear_output(wait=True)

          print("Profile Information")
          print(f"Name: {name_input.value}")
          print(f"Email: {email_input.value}")
          print("Career Interests:")
          print(interest_input.value)
          print("Work Experience:")
          print(experience_input.value)
          print("\nEducation:")
          print(education_input.value)
          print("\nSkills:")
          print(skills_input.value)

  submit_button.on_click(on_submit_click)

  form_widgets = [
      name_input,
      email_input,
      widgets.VBox([
          widgets.HTML("<b>Career Interests:</b>"),
          interest_input
      ]),
      widgets.VBox([
          widgets.HTML("<b>Work Experience (Details):</b>"),
          experience_input
      ]),
      widgets.VBox([
          widgets.HTML("<b>Education (Details):</b>"),
          education_input
      ]),
      widgets.VBox([
          widgets.HTML("<b>Skills (Keywords):</b>"),
          skills_input
      ]),
      submit_button,
      output_area
  ]

  full_form = widgets.VBox(form_widgets)
  display(full_form)


In [ ]:
# ##Resume upload
# from google.colab import files
# if profile_creation_method == 'Yes, I would like to upload my resume.':
#     print("Please upload your resume.")
#     resume = files.upload()

#     for fn in resume.keys():
#         print('User uploaded file "{name}" with length {length} bytes'.format(
#             name=fn, length=len(resume[fn])))

In [ ]:
##Resume parsing
##PDF only for now?



In [ ]:
##Company
company = input("What company are you messaging about? ")

##Role type
role_type = input("What type of role are you messaging about? ")

##Job title
job_title = input("What is the job title you are messaging about? ")

##Job description
job_description = input("What is the job description you are messaging about? ")

In [ ]:
##Message type selection
##Options: LinkedIn connection notes, InMails, Referral requests, Cover letters, Thank-you notes, Recruiter outreach emails
import ipywidgets as widgets
from IPython.display import display

menu_options = [
    'LinkedIn connection notes',
    'InMails',
    'Referral requests',
    'Cover letters',
    'Thank-you notes',
    'Recruiter outreach emails'
]

message_type_dropdown = widgets.Dropdown(
    options=menu_options,
    value= 'LinkedIn connection notes',
    description='Choose the type of message you want to write:',
    disabled=False,
)

display(message_type_dropdown)

selected_message_type = message_type_dropdown.value

In [ ]:
##Optional context: tone, structure, content
tone_description = input("Please specify any particular tone you want the message to have.")
structure_description = input("Please specify any particular structure you want the message to have.")
content_description = input("Please specify any particular content you want the message to have.")

## 3. Feature Extraction 30pts

---

Implement any existing feature extraction tools and methods (term frequency, word embeddings etc)

In [ ]:
!pip install python-dateutil newspaper3k GoogleNews pandas lxml_html_clean nltk transformers

In [ ]:
import time
import random
import requests
import pandas as pd
import nltk
nltk.download('punkt')
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.relativedelta import relativedelta
from newspaper import Article, Config
from GoogleNews import GoogleNews
from nltk.tokenize import sent_tokenize
from transformers import pipeline
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

In [ ]:
# configuration
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 30
company = "Microsoft" #TODO replace with dynamic company input
role = "Software Engineer" #TODO replace with dynamic role input

# search parameters 
# TODO create a list of company queries with different paremeters (e.g. business news, finance news, technology news, etc.) and cast a wider net of articles to gather
company_query = f"{company} Latest News"
end_date_str = datetime.now().strftime('%m/%d/%Y')
start_date_str = (datetime.now() - relativedelta(months=3)).strftime('%m/%d/%Y')

# establish request session
session = requests.Session()
retries = Retry(total=5, backoff_factor=1)
adapter = HTTPAdapter(max_retries=retries)
session.mount('http://', adapter)
session.mount('https://', adapter)

# search 5 pages of Google News results for the company's latest news
gn = GoogleNews(start=start_date_str, end=end_date_str)
gn.search(company_query)
pages_to_fetch = 5

news_items = []
seen_links = []
for page in range(1, pages_to_fetch + 1):
    gn.getpage(page)
    items = gn.results()
    if not items:
        continue
    for item in items:
        link = item.get('link')
        if not link or link in seen_links:
            continue
        seen_links.append(link)
        news_items.append(item)

# iterate over each article and parse using newspaper library
parsed_articles = []
failed_articles = []

for i, item in enumerate(news_items):
    article_link = item.get('link')
    
    headers = {
        "User-Agent": user_agent,
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/"
    }

    try:
        # get HTML content with retries
        resp = session.get(article_link, headers=headers, timeout=5)
        resp.raise_for_status()
        html = resp.text

        # give the HTML to newspaper to avoid its default downloader being blocked
        article = Article(article_link, config=config)
        article.download(input_html=html)
        article.parse()
        article.nlp()
        if not article.text and not article.title:
            continue

        parsed_articles.append({
            'Title': article.title,
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': article.summary,
            'Full_Text': article.text
        })

    except Exception as e:
        # store failed articles in separate list
        failed_articles.append({
            'Title': item.get('title', 'N/A'),
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': "Scraping failed.",
            'Full_Text': f"Error during parsing/download: {e}"
        })

    # add a random pause between 2 and 6 seconds to mitigate blocks/throttling 
    time_to_wait = random.uniform(2, 6)
    time.sleep(time_to_wait)

print("Articles parsed: ", len(parsed_articles))
df = pd.DataFrame(parsed_articles)
df.head()

In [ ]:
summarizer = pipeline("text2text-generation", model="google/flan-t5-large")

# summarize articles
summaries = []

for article in parsed_articles:
    text = article.get('Full_Text').strip()
    if not text:
        summaries.append((article.get('Link'),'Summary failure.'))

    article_sentences = sent_tokenize(text)
    
    prompt = f"Write a one sentence summary of the content contained in the following article; specifically highlight its impact or relevance on {company} in a context that would be useful for a job interview of a position as a {role} at the company:\n{text}"

    summary_result = summarizer(prompt,max_length=130)

    summaries.append((article.get('Link'), summary_result[0].get('generated_text')))


In [ ]:
print(summaries)

In [ ]:
# old scraping code
# configuration
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 30
company = "Microsoft" #TODO replace with dynamic company input
role = "Software Engineer" #TODO replace with dynamic role input

# search parameters 
company_query = f"{company} Latest News"
end_date_str = datetime.now().strftime('%m/%d/%Y')
start_date_str = (datetime.now() - relativedelta(months=3)).strftime('%m/%d/%Y')

gn = GoogleNews(start=start_date_str, end=end_date_str)
gn.search(company_query)
news_items = gn.results()
parsed_articles = []

# loop over news articles
for i, item in enumerate(news_items):
    article_link = item.get('link')

    try:
        article = Article(article_link, config=config)

        article.download()
        article.parse()
        article.nlp()

        if not article.text and not article.title:
            continue

        parsed_articles.append({
            'Title': article.title,
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': article.summary,
            'Full_Text': article.text
        })

    except Exception as e:
        # TODO refine error handling so that we don't stop until we have 10 articles (e.g. failed article doesn't count)
        parsed_articles.append({
            'Title': item.get('title', 'N/A'),
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': "Scraping failed.",
            'Full_Text': f"Error during parsing/download: {e}"
        })

    # add a random pause between 2 and 6 seconds to mitigate blocks/throttling 
    time_to_wait = random.uniform(2, 6)
    time.sleep(time_to_wait)

df = pd.DataFrame(parsed_articles)
df.head()

## 4. Main Functionality 10pts

---

This is your main task. For example, if you are creating a translation app, your main task is Translation.

In [ ]:
import time
import random
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from newspaper import Article, Config
from GoogleNews import GoogleNews # Use the alternative package

# --- Configuration (Keep robust settings) ---
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 30
config.memoize_articles = False # Avoid cache issues
company = "Microsoft" #TODO replace with dynamic company input
role = "Software Engineer" #TODO replace with dynamic role input

# --- Setup Search Parameters (assuming company is defined) ---
company_query = f"{company} Latest News"
end_date_str = datetime.now().strftime('%m/%d/%Y')
start_date_str = (datetime.now() - relativedelta(months=3)).strftime('%m/%d/%Y')

gn = GoogleNews(start=start_date_str, end=end_date_str)
gn.search(company_query)
news_items = gn.results()
parsed_articles = []

print(f"Starting scrape for {len(news_items)} articles. This will take a few minutes...")

# --- Loop with Sleep ---
for i, item in enumerate(news_items):
    article_link = item.get('link')

    try:
        article = Article(article_link, config=config)

        article.download()
        article.parse()
        article.nlp()

        if not article.text and not article.title:
            raise Exception("Article text and title were blank after parsing.")

        parsed_articles.append({
            'Title': article.title,
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': article.summary,
            'Full_Text': article.text
        })

    except Exception as e:
        # TODO refine error handling so that we don't stop until we have 10 articles (e.g. failed article doesn't count)
        parsed_articles.append({
            'Title': item.get('title', 'N/A'),
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': "Scraping failed.",
            'Full_Text': f"Error during parsing/download: {e}"
        })

    # Add a random pause between 2 and 6 seconds
    # This is CRITICAL for avoiding blocks/throttling
    time_to_wait = random.uniform(2, 6)
    time.sleep(time_to_wait)

    if (i + 1) % 10 == 0:
        print(f"Processed {i + 1} articles. Waiting...")

df = pd.DataFrame(parsed_articles)
df.head()

In [ ]:

nltk.download('punkt')

from nltk.tokenize import sent_tokenize
from transformers import pipeline

# TODO pick pipeline model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

##Summarize articles
summaries = []

for article in parsed_articles:
    text = article.get('Full_Text').strip()
    if not text:
        summaries.append((article.get('Link'),'Summary failure.'))

    article_sentences = sent_tokenize(text)
    
    prompt = f"Write a one sentence summary for the following article, specifically related to its impact or relevance on {company} in a context that may be useful for a job interview of a position as a {role}:\n{text}"

    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    summary_result = summarizer(prompt)
    print(summary_result)
    print(summary_result[0])
    print(summary_result[0].get('summary_text'))

    if '.' in summary_result['summary_text']:
        summaries.append((article.get('Link'), summary_result[0].get('summary_text').split('. ')))
    else:
        summaries.append((article.get('Link'), summary_result[0].get('summary_text')))

print(summaries)

In [ ]:
##Generate message


## 5. Personal Contribution Statement 10pts

---

*   Summary of tasks and team members' contributions
*   Proofreading




**Our assignments for Part 2...**

1. PDF parsing + turn into profile - **Cassie**
	* Offer parsed resume as a form where they can correct values
2. News scraping (+ Reddit https://www.reddit.com/dev/api/) + summarization - **Brendan**
	* Offer filters for topics in news
3. Prompted engineering for messages (including offering news highlights to choose from + maybe reddit) + code to generate message - **Anshika**
	* Who to send to form: Who? How do you know them?
	* Creativity (temperature) measure?

***** Don't forget to put in your specific feedback! *****